# Speckle Tracking

In [ ]:
from pathlib import Path
# from project_heart.lv import LV
from project_heart.utils import set_jupyter_backend
from project_heart.enums import *
set_jupyter_backend("pythreejs")
import logging
import numpy as np
float_formatter = "{:.5f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})

from project_heart.examples import get_lv_ideal
from project_heart.examples import get_lv_typeA

lv_ideal = get_lv_ideal()
lv_typeA = get_lv_typeA()

sample_spk_typeA_endo = lv_typeA.get_speckles(spk_name="SAMPLE", spk_group="endo", spk_collection="SAMPLE")[0]
sample_spk_typeA_epi = lv_typeA.get_speckles(spk_name="SAMPLE", spk_group="epi", spk_collection="SAMPLE")[0]

## What are Speckles?

Speckles ...

In [ ]:
a = np.radians(0.0)
normal = [np.cos(a),np.sin(a),0.0]
# from project_heart.utils.vector_utils import project_vec_a_onto_vec_b
# use_normal = project_vec_a_onto_vec_b(normal, lv_typeA.get_normal())
# use_normal = align_vec_axis(normal, [0,0,1], lv_typeA.get_normal(), False)
# print(use_normal, normal)
use_normal = normal
n_subsets = 6
_ = lv_typeA.create_speckles(
            collection="SAMPLETEST2",
            group="epi",
            name="SAMPLE-LONG",
            from_nodeset=LV_SURFS.EPI,
            exclude_nodeset=[LV_SURFS.BASE, LV_SURFS.ENDO],
            d=2.5,
            k=0.5,
            normal_to=use_normal,
            n_subsets=n_subsets,
            subsets_criteria="z2",
            cluster_criteria="angles3",
            n_clusters=18,
            t=0.0,
            kmin=-1,
            kmax=0.85,
            log_level=logging.WARN,
        )

sample_spk_typeA = lv_typeA.get_speckles(spk_name="SAMPLE-LONG", spk_group="epi", spk_collection="SAMPLETEST2")

plotter = lv_typeA.plot_speckles(sample_spk_typeA, 
                       cmap="hot",
                       point_size=300,
                       add_k_centers=True, 
                       k_centers_as_line=True, 
                       k_bins=True,
                       )

In [ ]:
a = np.radians(90.0)
normal = [np.cos(a),np.sin(a),0.0]
# from project_heart.utils.vector_utils import project_vec_a_onto_vec_b
# use_normal = project_vec_a_onto_vec_b(normal, lv_typeA.get_normal())
# use_normal = align_vec_axis(normal, [0,0,1], lv_typeA.get_normal(), False)
# print(use_normal, normal)
# use_normal = normal
n_subsets = 0
_ = lv_typeA.create_speckles(
            collection="SAMPLETEST25",
            group="epi",
            name="SAMPLE-LONG",
            from_nodeset=LV_SURFS.EPI,
            exclude_nodeset=[LV_SURFS.BASE, LV_SURFS.ENDO],
            d=2.5,
            k=0.5,
            normal_to=normal,
            n_subsets=n_subsets,
            subsets_criteria="z2",
            cluster_criteria="angles3",
            n_clusters=18,
            t=0.0,
            kmin=-1,
            kmax=0.85,
            log_level=logging.WARN,
        )

sample_spk_typeA = lv_typeA.get_speckles(spk_name="SAMPLE-LONG", spk_group="epi", spk_collection="SAMPLETEST25")

plotter = lv_typeA.plot_speckles(sample_spk_typeA, 
                       cmap="hot",
                       point_size=300,
                       add_k_centers=True, 
                       k_centers_as_line=True, 
                       k_bins=True,
                       )

In [ ]:
normal = [0,0,1]
n_subsets = 6
_ = lv_typeA.create_speckles(
            collection="SAMPLETEST3",
            group="epi",
            name="SAMPLE-LONG",
            from_nodeset=LV_SURFS.EPI,
            exclude_nodeset=[LV_SURFS.BASE, LV_SURFS.ENDO],
            d=4.0,
            k=0.5,
            normal_to=normal,
            n_subsets=n_subsets,
            subsets_criteria="angles",
            cluster_criteria="angles3",
            n_clusters=18,
            t=0.0,
            kmin=-1,
            kmax=0.85,
            log_level=logging.WARN,
        )

sample_spk_typeA = lv_typeA.get_speckles(spk_name="SAMPLE-LONG", spk_group="epi", spk_collection="SAMPLETEST3")

plotter = lv_typeA.plot_speckles(sample_spk_typeA, 
                       cmap="hot",
                       point_size=300,
                       add_k_centers=True, 
                       k_centers_as_line=True, 
                       k_bins=True,
                    #    k_center_filters=dict(
                    #        mfilter_ws=3, 
                    #        sfilter_ws=5, 
                    #        sfilter_or=1, 
                    #        keep_first=True, 
                    #        keep_last=True)
                       )

In [ ]:
from project_heart.utils.cloud_ops import relate_closest, map_A_to_B

In [ ]:
from sklearn.model_selection import StratifiedKFold, KFold

In [ ]:
sample_spk_typeA = lv_typeA.get_speckles(spk_group="epi", spk_collection="long-1")
sample_spk_typeA

In [ ]:
# sample_spk_typeA = lv_typeA.get_speckles(spk_name="TESTE", spk_group="endo", spk_collection="TESTELONG1312-1")
lv_typeA.plot_speckles(sample_spk_typeA[5], 
                       cmap="hot", 
                        point_size=200,
                       add_k_centers=True, 
                    #    k_centers_as_line=True, 
                       k_bins=True,
                           k_center_filters=dict(
                           mfilter_ws=5, 
                           sfilter_ws=9, 
                           sfilter_or=1, 
                           keep_first=False, 
                           keep_last=False)
                       )

In [ ]:
spk = lv_typeA.create_speckles(
            collection="TESTELONG-1",
            group="endo",
            name="TESTE",
            from_nodeset=LV_SURFS.ENDO,
            exclude_nodeset=LV_SURFS.BASE, # does not afect ideal case
            d=1.75,
            k=0.5,
            normal_to=[1,0,0],
            n_subsets=0,
            subsets_criteria="z2",
            cluster_criteria="angles3",
            n_clusters=80,
            t=0.0,
            kmin=-1,
            kmax=0.92,
            log_level=logging.DEBUG,
        )

In [ ]:
sample_spk_typeA = lv_typeA.get_speckles(spk_name="TESTE", spk_group="endo", spk_collection="TESTELONG-1")
lv_typeA.plot_speckles(sample_spk_typeA, 
                       cmap="hot", 
                        point_size=200,
                       add_k_centers=True, 
                    #    k_centers_as_line=True, 
                       k_bins=True,
                           k_center_filters=dict(
                           mfilter_ws=5, 
                           sfilter_ws=9, 
                           sfilter_or=1, 
                           keep_first=False, 
                           keep_last=False)
                       )

In [ ]:

from project_heart.utils.vector_utils import *
# a = np.radians(90.0)
# normal = np.array([np.cos(a),np.sin(a),0.0])
# normal = np.array([0.0,0.0,1.0])
normal = sample_spk_typeA[-1].normal

n_subsets = 1
n_cluster = 60
check_orientation = False if n_subsets > 1 else True

_ = lv_typeA.create_speckles(
            collection="SAMPLET348543",
            group="endo",
            name="SAMPLE-LONG",
            from_nodeset=LV_SURFS.ENDO,
            exclude_nodeset=LV_SURFS.BASE,
            d=2.4,
            k=0.5,
            normal_to=normal,
            n_subsets=n_subsets,
            # subsets_criteria="z2",
            # subsets_criteria="angles",
            # cluster_criteria="z2",
            n_clusters=0,
            t=0.0,
            kmin=-1,
            kmax=0.92,
            log_level=logging.WARN,
        )

sample_spk_typeA = lv_typeA.get_speckles(spk_name="SAMPLE-LONG", spk_group="epi", spk_collection="SAMPLET348543")
for i in range(n_subsets):
    
    spk = sample_spk_typeA[i]
    ref_center = spk.center
    pts = lv_typeA.points(mask=spk.ids)

    import logging
    logger = logging.getLogger("test")
    logger.setLevel(logging.DEBUG)
    logger.debug("i: {}.".format(i))
    
    if normal[2] < 0:
        normal = -normal
        logger.debug("using opposite normal for computation as negative "
                     "(on z) axis are not fully tested: {}.".format(normal))
    # project points onto a single plane based on spk normal
    logger.debug("pts.shape: {}.".format(pts.shape))
    logger.debug("pts[:5]: \n{}.".format(pts[:5]))
    plane_d = calc_plane_d(normal, ref_center)
    logger.debug("plane_d: {}.".format(plane_d))
    ppts = project_pts_onto_plane(pts, normal, plane_d)
    p_center = project_pts_onto_plane(ref_center, normal, plane_d)[0]
    logger.debug("p_center: {}.".format(p_center))
    # compute angles between a reference vector and other vectos
    vecs = ppts - p_center 
    vecs_sum = np.linalg.norm(vecs, axis=1)
    ref_max_pt = np.argmax(vecs_sum)
    ref_min_pt = np.argmin(vecs_sum)
    logger.debug("ref_max_pt: {}.".format(ref_max_pt))
    logger.debug("ref_max_pt: {}.".format(ref_min_pt))
    ref_max_vec = vecs[ref_max_pt]
    ref_min_vec = vecs[ref_min_pt]
    logger.debug("ref_max_vec: {}.".format(ref_max_vec))
    logger.debug("ref_min_vec: {}.".format(ref_min_vec))
    # compute angles based on 
    if check_orientation:
        logger.debug("Using a none or single subset, meaning large speckle. \n" 
                     "-> 'check_orientation' is set to True."
                     "-> Will try to eliminate possible discontinuities.")
        angles = angle_between(vecs, ref_max_vec, check_orientation=True, zaxis=normal)    
        search_for_jumps = True
        search_trial = 0
        max_search_trials = 25
        dist_step = 0.25
        while search_for_jumps:
            logger.debug("Searching for discontinuity... {}".format(search_trial+1))
            sort_ids = np.argsort(angles)
            ppts_sorted_by_angles = ppts[sort_ids]
            dists_ppts = np.linalg.norm(ppts_sorted_by_angles[1:] - ppts_sorted_by_angles[:-1], axis=1)
            dists_grad = np.gradient(dists_ppts)
            d_mean, d_stdev = np.mean(dists_grad), np.std(dists_grad)
            dist_jump = np.where(np.abs(dists_grad - d_mean) > (8 + (search_trial*dist_step)) *d_stdev)[0]
            if len(dist_jump) > 0:
                logger.debug("Discontinuity Found: {}".format(len(dist_jump)))
                dist_jump_val = dists_grad[dist_jump]
                logger.debug("dist_jump: {} -> {}.".format(dist_jump, dist_jump_val))
                new_max_pt = sort_ids[dist_jump[np.argmax(dist_jump_val)]+1]
                ref_max_vec = vecs[new_max_pt]
                ref_max_pt = new_max_pt
                angles = angle_between(vecs, ref_max_vec, check_orientation=True, zaxis=normal)
            else:
                logger.debug("No discontinuity found: {}".format(len(dist_jump)))
                search_for_jumps = False
            if search_trial >= max_search_trials:
                logger.debug("Could not resolve jumps under {} trials. \n"
                             "-> Continuing (errors might occur)".format(max_search_trials))
                search_for_jumps = False
            search_trial += 1
    else:
        logger.debug("Using angles without orientation [0, 180]")
        angles = angle_between(vecs, ref_max_vec, check_orientation=False)
    # get reference vecs for binarization bounds
    min_a, max_a = np.min(angles), np.max(angles)
    logger.debug("min_a: {}.".format(np.degrees(min_a)))
    logger.debug("max_a: {}.".format(np.degrees(max_a)))
    min_idx, max_idx = np.argmin(angles), np.argmax(angles)
    ref_max_vec = vecs[max_idx]
    ref_min_vec = vecs[min_idx]
    logger.debug("Updating reference vectors.")
    logger.debug("ref_max_vec: {}.".format(ref_max_vec))
    logger.debug("ref_min_vec: {}.".format(ref_min_vec))
    # create bins
    bins = np.digitize(
        angles, np.linspace(min_a*0.999, max_a*1.001, n_cluster+1))
    logger.debug("Number of bins found: {}.".format(len(np.unique(bins))))
    min_bin_id, max_bin_id = np.min(bins), np.max(bins)
    logger.debug("Min, Max bin Ids: [{}, {}].".format(min_bin_id, max_bin_id))
    # update reference vectors (make sure we have extreme pts)
    min_mask = np.where(bins==min_bin_id)[0]
    max_mask = np.where(bins==max_bin_id)[0]
    ref_last_bin_idx = max_mask[np.argmax(angles[max_mask])]
    ref_first_bin_idx = min_mask[np.argmin(angles[min_mask])]
    logger.debug("ref_last_bin_idx: {}.".format(ref_last_bin_idx))
    logger.debug("ref_first_bin_idx: {}.".format(ref_first_bin_idx))
    ref_last_vec = vecs[ref_last_bin_idx]
    ref_first_vec = vecs[ref_first_bin_idx] 
    logger.debug("ref_last_vec: {}.".format(ref_last_vec))
    logger.debug("ref_first_vec: {}.".format(ref_first_vec))
    # compute vector normal and cross vectors
    first_cross_last = np.cross(unit_vector(ref_first_vec), unit_vector(ref_last_vec)) 
    cross_normal = np.degrees(angle_between(normal, first_cross_last, check_orientation=False)) 
    first_last_angle = np.degrees(angle_between(ref_first_vec, ref_last_vec, check_orientation=True, zaxis=normal))
    logger.debug("normal: {}.".format(normal))
    logger.debug("first_cross_last: {}.".format(first_cross_last))
    logger.debug("cross_normal: {}.".format(cross_normal))
    logger.debug("first_last_angle: {}.".format(first_last_angle))
    # recompute bins to ensure we have full range (min to max vecs)
    # a_normal_Zaxis = np.degrees(angle_between(normal, [0.0,0.0,1.0], check_orientation=False))
    if not check_orientation:# and a_normal_Zaxis < 45: 
        # logger.debug("Re-computing bins as angle from normal to Z axis < 45: {}.".format(a_normal_Zaxis))
        logger.debug("Recomputing bins based on new vecs.")
        angles = angle_between(vecs, ref_last_vec, check_orientation=False)
        min_a, max_a = np.min(angles), np.max(angles)
        logger.debug("min_a: {}.".format(np.degrees(min_a)))
        logger.debug("max_a: {}.".format(np.degrees(max_a)))
        bins = np.digitize(
            angles, np.linspace(min_a*0.999, max_a*1.001, n_cluster+1))
        # check gradients between gradient centers -> angle between first to other bins
        # we want ccw bins                
        centers = np.asarray([np.mean(ppts[bins==bin_id], axis=0) for bin_id in sorted(np.unique(bins))], dtype=np.int64)
        c_vecs = centers - p_center
        c_angles = np.degrees(angle_between(c_vecs, c_vecs[0], check_orientation=True, zaxis=normal))
        c_grads = np.gradient(c_angles[1:])
        mean_c_grads = np.mean(c_grads)
        logger.debug("mean_c_grads: {}.".format(mean_c_grads))
        # check for reversion
        should_reverse = False
        a_normal_Xaxis = np.round(np.degrees(angle_between(normal, [1.0,0.0,0.0], check_orientation=False)), 3)
        if a_normal_Xaxis < 45:
            should_reverse = not should_reverse
            logger.debug("Flip should_reverse due normal orientation -> [{}]: a_normal_Xaxis: {}.".format(should_reverse, a_normal_Xaxis))   
        # a_normal_Zaxis = np.round(np.degrees(angle_between(normal, [0.0,0.0,1.0], check_orientation=False)), 3)
        # if a_normal_Zaxis < 45:
        #     should_reverse = not should_reverse
        #     logger.debug("Flip should_reverse due normal orientation -> [{}]: a_normal_Zaxis: {}.".format(should_reverse, a_normal_Zaxis))   
        # if cross_normal < 90 and mean_c_grads > 0:
        #     should_reverse = not should_reverse
        #     logger.debug("bins should reverse [{}] as cross_normal < 90 and mean_c_grads > 0.".format(should_reverse))
        # if cross_normal < 90 and first_last_angle > 180:
        #     should_reverse = not should_reverse
        #     logger.debug("bins should reverse [{}] as cross_normal < 0 and first_last_angle > 180.".format(should_reverse))
        # elif cross_normal > 90 and first_last_angle > 180: #:
        #     should_reverse = not should_reverse
        #     logger.debug("bins should reverse [{}] as cross_normal > 0, first_last_angle > 180 and mean_c_grads > 0.".format(should_reverse))
        if first_last_angle > 180:
            should_reverse = not should_reverse
            logger.debug("bins should reverse [{}] as first_last_angle > 180 ".format(should_reverse))
        # if algorithm decided to reverse, reverse bins
        if should_reverse:
            logger.debug("reversing bins.")
            uvals = np.unique(bins)
            logger.debug("uvals: {}.".format(uvals))
            new_bins = np.zeros(len(bins), dtype=np.int64)
            for i, u in enumerate(uvals[::-1]):
                new_bins[bins==u] = i
            bins = new_bins + 1
      
    
    plotter = lv_typeA.plot_speckles(spk, 
                        cmap=None,
                        color="blue",
                        point_size=100,
                        add_k_centers=False, 
                        k_centers_as_line=True, 
                        k_bins=False,
                        re = True
                        )
    
    plotter.add_lines(np.vstack((p_center, p_center+5*normal)), color="magenta")
    plotter.add_lines(np.vstack((p_center, p_center+5*first_cross_last)), color="orange")
    
    
    plotter.add_lines(np.vstack((p_center, ppts[ref_max_pt])), color="blue")
    plotter.add_lines(np.vstack((p_center, ppts[ref_min_pt])), color="green")
    
    plotter.add_lines(np.vstack((p_center, ppts[ref_last_bin_idx])), color="cyan")
    plotter.add_lines(np.vstack((p_center, ppts[ref_first_bin_idx])), color="brown")
        
    plotter.add_points(p_center, color="green", point_size=240)
    plotter.add_points(ppts, color="magenta", point_size=200)
    plotter.add_points(pts, scalars=bins-1, categories=True, cmap="hot", point_size=240)
    plotter.show(window_size=(600,400))

In [ ]:
np.cross(normal, -lv_typeA.get_normal())

## Sample Speckles

### Longitudinal Speckles

In [ ]:
# lv_ideal.plot_speckles(endo_long[1], add_centers=True, cmap="tab10", categories=True)

In [ ]:
# lv_ideal.plot_speckles(epi_long[0], cmap="jet", add_k_centers=True, k_centers_as_line=True, k_bins=True, categories=True)

### Circumferential Speckles

In [ ]:
# lv_ideal.plot_speckles(endo_circ[7], add_centers=True, cmap="tab10", categories=True)

In [ ]:
# lv_ideal.plot_speckles(epi_circ[7], cmap="jet", add_k_centers=True, k_centers_as_line=True, k_bins=True, categories=True)